In [1]:
def a_method(a, b)
  a + yield(a, b)
end

a_method(1, 2) {|x, y| (x + y) * 3}

10

只有在调用一个方法时，才可以定义一个块。块会被直接传递给这次方法，该方法可以用`yield`关键字调用这个块。

块可以拥有自己的参数，比如上面例子中的`x`和`y`。当回调块时，你可以像调用方法那样为块提供参数。另外，像方法一样，块的最后一行代码执行的结果会被作为返回值。

在一个方法里，你可以询问当前的方法调用是后包含块。这可以通过`Kernel#block_given?`方法做到

In [2]:
def a_method
  return yield if block_given?
  'no block'
end

p a_method
p a_method {"here's a block!"}

"no block"
"here's a block!"


"here's a block!"

In [3]:
module Kernel
  def with(resource)
    begin
      yield
    ensure
      resource.dispose
    end
  end
end

:with

定义一个块的时候，它会获取环境中的绑定。当块被传给一个方法时，它会带着这些绑定给一块进入方法

In [4]:
def my_method
  x = "Goodbye"
  yield("cruel")
end

x = "hello"
my_method {|y| "#{x}, #{y} wolrd"}

"hello, cruel wolrd"

创建代码块时候，你会获得局部绑定（比如上面的`x`），然后把代码块连同它的绑定传给一个方法。在上面的例子中，代码块的绑定中包括一个名为`x`的变量。虽然在方法中也定义了一个变量`x`，但代码块看到的`x`还是在代码块定义时绑定的`x`，方法中的`x`对于这个代码块来说是不可以见的。

还可以在代码块内定义额外的绑定，但这些绑定在代码块结束时就消失了。

In [5]:
def just_yield
  yield
end

top_level_variable = 1

just_yield do
  top_level_variable += 1
  local_to_block = 1
end

p top_level_variable
local_to_block

2


NameError: undefined local variable or method `local_to_block' for main:Object

基于这样的特性，人们喜欢把代码块称为闭包(`closure`)。换句话说，代码块可以获取局部绑定，并一直带着他们。

下面的例子演示了程序运行时作用域是如何切换的，`Kernel#local_variable`方法用来跟踪绑定的名字：

In [6]:
v1 = 1
class MyClass
  v2 = 2
  p local_variables
  def my_method
    v3 = 3
    p local_variables
  end
  p local_variables
end

obj = MyClass.new
obj.my_method
obj.my_method
local_variables

[:v2]
[:v2]
[:v3]
[:v3]


[:v1, :obj, :top_level_variable, :_i4, :_4, :x, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :_pry_, :_out_, :_in_, :_oh, :_ih, :title]

在某些语言(如`Java`和`C#`中)，有“内部作用域(inner scope)”的概念。在内部作用域里可以看到”外部作用域”的变量。但`ruby`没有嵌套式的作用域，它的作用域是截然分开的：一旦进入一个新的作用域，原先的绑定会被被替换为一组新的绑定。这意味着在程序进入`MyClass`之后，`v1`就落在了作用域之外，从而不可见了。

在i定义`MyClass`的作用域中，程序定义了`v2`以及一个方法。因为方法中的代码还没有被执行，所以直到类定义结束前，程序不会再打开一个新的作用域。在方法定义完成之后，用`class`关键字打开的作用域会永远关闭，同时程序回到顶级作用域。

全局变量可以在 **任何作用域**访问

In [7]:
def a_scope
  $var = "some value"
end

def another_scope
  $var
end


:another_scope

In [8]:
a_scope
another_scope

"some value"

谁都可以设置全局变量，你几乎没办法确定是谁修改了他们。因此，能不使用全局变量，就i尽量不要使用。有时可以使用顶级实例变量代替全局变量。它们是顶级对象`main`的实例变量。

In [9]:
@var  = "The top level @var"

def my_method
  @var
end

my_method

"The top level @var"

In [10]:
class MyClass2
  def my_method
    @var = "This is not the top-level @var!"
    $var
    p @var
    p $var
  end
end

myclass = MyClass2.new
myclass.my_method

"This is not the top-level @var!"
"some value"


"some value"

准备地说，程序会在三个地方关闭一个作用域，同事打开一个新的作用域

- 类定义
- 模块定义
- 方法

每当程序进入（或离开）类定义、模块方法、方法时，就会发生作用域切换。这三种情况分别以`class`、`module`和`def`关键字，就会发生作用域切换。这三种情况分别以`class`、`module`和`def`关键字作为标志。每个关键字都对应一个作用域门(`Scope Gate`)。

In [11]:
v1 = 1
class MyClass
  v2 = 2
  local_variables
  def my _method
    v3 = 3
    local_variables
  end
  local_variables
end

obj = MyClass.new
p obj.my_method
local_variables

[:v3]
[:v3]


[:_i9, :_9, :myclass, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :v1, :obj, :top_level_variable, :_i4, :_4, :x, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :_pry_, :_out_, :_in_, :_oh, :_ih, :title]

在`class/module`与`def`之间还有一个微妙的区别。在类定义和模块定义中的代码会立即执行。相反，在方法定义的代码不会立即执行。

In [12]:
my_var = "Success"

MyClass = Class.new do
  puts "#{my_var} in the class definition"
  
  define_method :my_method do
    "#{my_var} in the method"
  end
end

MyClass.new.my_method

Success in the class definition


(pry):100: warning: already initialized constant MyClass
(pry):43: warning: previous definition of MyClass was here


"Success in the method"

使用方法调用来替代作用域门，就可以然给一个作用域看到另外一个作用域里的变量。这种技巧称为嵌套文法作用域（`nested lexical scopes`），不过不过很多`Ruby`程序员习惯叫它“扁平化作用域”，表示如果两个作用域挤压在一起，它们就可以共享的变量。也可以简称为**扁平作用域**。

In [13]:
def define_methods
  shared = 0
  Kernel.send :define_method, :counter do
    shared
  end
  
  Kernel.send :define_method, :inc do
    shared += x
  end
end
define_methods

counter
inc(4)
counter

ArgumentError: wrong number of arguments (given 1, expected 0)

这个例子定义了两个**内核方法**，它还使用动态派发法术来访问`Kernel`的私有类方法`defind_method`。`Kernel#counter`和`Kernel#inc`方法都可以看到`shared`变量，但是其他方法却看不见它，因为这个变量被作用域门保护着，这就是使用`define_methods`方法的原因。这种用来共享变量的技巧称为共享作用域。

每个Ruby作用域都包换一组绑定。不同的作用域之间被作用域门（`class`、`module`、`def`）分割开来。

要想让某个绑定穿越作用域，可以使用代码块。一个代码块是一个闭包，当定义一个代码块时，它会捕获当前环境中的绑定,并把这个闭包传递给该方法

可以使用`Class.new`方法代替`class`关键字，用`Module.new`方法代替`module`关键字，用`Module#define_method`方法代替`def`关键字。这就是扁平作用域的，它更是闭包上常用的法术。

如果一个扁平作用域中定义了多个方法，把这些方法用一个作用域门保护起来，它们就可以共享绑定，这种技巧称为共享作用域

In [ ]:
class InstanceVal
  def initialize
    @v = 1
  end
end

obj = InstanceVal.new

obj.instance_eval do
  self
  @v
end

运行时，代码块的接收者会成为`self`，因此它可以访问接收者的私有方法和实例变量（如`@v`）。因此他可以访问接收者的私有变量（如`@v`）。即使`instance_eval`方法修改了`self`变量，传给`instance_eval`方法的代码块依然可以看到在它定义时的那些绑定，就像其他代码块那样：

In [16]:
v = 2
obj.instance_eval {@v  = v}
obj.instance_eval {@v}

2

上面的三行代码在同一个扁平作用域中执行，因此它们都可以访问局部变量`v`。由于代码块把它的对象作为`self`，所有它们都可以访问`obj`的实例变量`@v`。我们把传递给`instance_eval`方法的代码块称为**上下文探针**，因为它就像是一个深入到对象中的代码片段，并可以对那个对象进行操作。

In [18]:
class C
  def initialize
    @x = 1
  end
end

class D
  def twisted_method
    @y = 2
    C.new.instance_eval {"@x: #{@x}， @y: #{@y}"}
  end
end

D.new.twisted_method

"@x: 1， @y: "

你可能认为`D#twisted_method`中的代码块既可以访问`C`类的实例变量，也可以访问`D`类中的`@y`实例变量，因为他们都在同一个**扁平作用域**里。然而，实例变量是依赖于当前对象`self`的。因此当`instance_eval`方法吧接收者变为当前对象`self`时，调用者的实例变量就落在作用于范围外了。这样，代码块中的`@y`被当成是`C`类的实例变量，此时它并未初始化。因此值还是`nil`。

In [20]:
class D
  def twisted_method
    @y = 2
    C.new.instance_exec(@y) {|y| "@x #{@x}, @y:#{y}"}
  end
end

:twisted_method

In [21]:
D.new.twisted_method

"@xL 1, @y:2"